In [ ]:
from rdflib import Namespace, Graph
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library

This document walks through how to write some simple templates


A template is a function that generates a graph. The template definition provides the structure of the graph and allows the content of the graph to be determined in part through the use of *parameters*. Parameters have a name (typically short and descriptive) and can be required or optional. Parameters are identified by their prefix: `urn:___param___#`

Templates are most easily written as YAML documents. A YAML file can contain more than one template. The names of the templates are at the top level of the YAML file as "keys".  The associated values are the content of the template. Templates can have 3 fields:
- `body`: a turtle-serialized graph defining the structure of what will be generated
- `optional`: a list of parameters which are optional in the template evaluation. If not bound, they will be removed from the generated graph
- `dependencies`: a list of dependencies on other templates

To start, let's look at a simple example without optional params or dependencies:

## Basic Template Anatomy

```yaml
my-thermostat:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasLocation P:room .
```

This template has two parameters: `name` and `room`. **All templates are required to have a `name` parameter.** This acts as the "root" of the template.

The `my-thermostat` template describes the thermostat as having a location, but no further properties. We also don't know what kind of thing `P:room` should be bound to when the template is used. Let's address the second problem first by introducing a dependency

```yaml
my-thermostat:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasLocation P:room .
  dependencies:
  - template: https://brickschema.org/schema/Brick#Room
    library: https://brickschema.org/schema/1.3/Brick
    args: {"name": "room"}
```

The `dependencies` key contains a list of documents which describe the dependencies of this template on other templates. The single entry above states that the `my-thermostat` template is dependent upon the `https://brickschema.org/schema/Brick#Room` template (automatically produced by importing the Brick ontology as a library), and the `name` parameter of the `Room` template is bound to the value of the `room` parameter in this template.

Let's now add another template to our library, which `my-thermostat` will also depend on, which defines some of the points we want to see on our thermostat:

```yaml
my-tstat-points:
  body: >
     @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasPoint P:temp, P:sp, P:co2 .
  dependencies:
    - template: https://brickschema.org/schema/Brick#Temperature_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "temp"}
    - template: https://brickschema.org/schema/Brick#Temperature_Setpoint
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "sp"}
    - template: https://brickschema.org/schema/Brick#CO2_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "co2"}
```

(There is syntax sugar for the above pattern)

We can now have our original `my-thermostat` template *depend* on this template. By binding this template to the `name` paramter of our original template, we are essentially composing the two templates together.

The full template library is now as follows:

```yaml
my-thermostat:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasLocation P:room .
  dependencies:
  - template: https://brickschema.org/schema/Brick#Room
    library: https://brickschema.org/schema/1.3/Brick
    args: {"name": "room"}
  - template: my-tstat-points
    args: {"name": "name"}
    
my-tstat-points:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasPoint P:temp, P:sp, P:co2 .
  dependencies:
    - template: https://brickschema.org/schema/Brick#Temperature_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "temp"}
    - template: https://brickschema.org/schema/Brick#Temperature_Setpoint
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "sp"}
    - template: https://brickschema.org/schema/Brick#CO2_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "co2"}
```

## Saving and Organizing Templates 

Templates are stored in the `YAML` format, which typically has a file suffix of `.yml` or `.yaml`

Templates can be grouped into a `Library` by placing all of the templates in the same directory. The name of that directory is treated as the name of the Library. You can have as many YAML files inside the directory as you want, and the organization of templates among those YAML files can follow any principle you like.

A few of our existing libraries follow different organizational principles:
- The *Guideline 36* library divides templates into different files depending on the system configuration they correspond to
- The *223P Components* library divides templates into different files depending on the kind of subsystem they are fore

You can also organize templates by equipment manufacturer, application profiles, and so on.

---

For example, if we saved the YAML documenbt above into a `thermostat.yaml` file and placed that file into a `my-templates` directory, the file structure would look like this:

```
$ tree my-templates
my-templates/
└── thermostat.yaml
```

and the `my-templates` library (and the `my-thermostat` template) could be loaded into BuildingMOTIF as follows:

```python
lib = Library.load(directory="my-templates")
tstat_templ = lib.get_template_by_name("my-thermostat")
```